**In this notebook I have implementented voice to text recognition on a medical dataset.
However due to insufficient audio dataset the 1D convolutional and LSTM models could not be tested for accuracy.
This notbook only focuses on the approach taken for voice to text recognition.
I have gathered audio files converted them to .WAV format at 8000 sampling rate, MONO channel.
Maximum size of the file is 3 sec and the files shorter than that have been padded.

In [1]:
import tensorflow as tf

In [2]:
from tensorflow import keras

In [28]:
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import utils
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D, LSTM, BatchNormalization,GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K

from tensorflow.keras.models import Sequential


In [4]:
import os
path = 'F:/Speech2Text/train_data'
os.listdir(path)

['Abdominoplasty',
 'Acanthamoeba',
 'Achondroplasia',
 'Addison_Disease',
 'Adenocarcinoma',
 'Cystic_Fibrosis',
 'Diabetes',
 'Gastritis',
 'Gastroenterostomy',
 'Gynecomastia',
 'Heartburn',
 'Hypertension',
 'Jugular_Vein',
 'Leucoderma',
 'Macular_Degeneration',
 'Mean_Corpuscular_Volume',
 'Meningitis_Contagious',
 'Meningoencephalitis_Toxoplasma',
 'Migraine',
 'Mitral_Valve_Prolapse',
 'Multiple_Sclerosis',
 'Ophthalmology',
 'Peroneal_muscle',
 'Posterior_muscles',
 'Pulmonary_artery']

In [5]:
labels=os.listdir(path)

**Find the count of each label

In [6]:
recordings=[]
for label in labels:
    waves = [f for f in os.listdir(path + '/'+ label) if f.endswith('.wav')]
    recordings.append(len(waves))
print(recordings)

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


**Find the sampling rate of labels**

In [7]:
for label in labels:
    #print(label)
    waves = [f for f in os.listdir(path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        samples, sample_rate = librosa.load(path + '/' + label + '/' + wav, sr = None)

In [8]:
print(sample_rate)
#print(samples)
samples = np.asarray(samples)
print(samples.ndim)

8000
1


**Feature Extraction.
Here Zero padding is done to the audio files who are less than maximum size of the file.
Next mfcc mel-freq features across 20 dimensions have been extracted.

In [9]:
sr = 8000
hop_length = int(0.5*sr + 2)
n_fft = hop_length + int(0.02*sr) #Consider 5ms window around the edges as a buffer/delay
#hop_length = n_fft//2
all_wave = []
all_label = []
max_size = 24800
for label in labels:
    #print(label)
    waves = [f for f in os.listdir(path + '/'+ label) if f.endswith('.wav')]
    for wav in waves:
        samples, sr = librosa.load(path + '/' + label + '/' + wav, sr=8000)
        size = samples.shape[0]
        
        n_pad = max_size - size
        samples = np.pad(samples, (0, n_pad), mode='constant')
        mfc = librosa.feature.mfcc(y=samples, sr=sr, n_mfcc=20, n_fft=n_fft, hop_length=hop_length).T
        #mfc = librosa.feature.mfcc(y=samples, sr=sr, n_mfcc=20).T
         
        all_wave.append(mfc)
        all_label.append(label)

In [10]:
# 24800 / 8000
print(sr)
print(hop_length)
print(n_fft)

8000
4002
4162


In [11]:
#print(all_wave)

In [12]:
all_wave = np.array(all_wave)
#all_wave = np.asarray(all_wave)
print(all_wave.shape)

(75, 7, 20)


In [13]:
#all_wave = np.expand_dims(all_wave, -1)
print(all_wave.dtype)

#print(all_wave.shape)

float32


**Label Encoding

In [14]:
le = LabelEncoder()
y=le.fit_transform(all_label)
classes= list(le.classes_)
print(classes)

['Abdominoplasty', 'Acanthamoeba', 'Achondroplasia', 'Addison_Disease', 'Adenocarcinoma', 'Cystic_Fibrosis', 'Diabetes', 'Gastritis', 'Gastroenterostomy', 'Gynecomastia', 'Heartburn', 'Hypertension', 'Jugular_Vein', 'Leucoderma', 'Macular_Degeneration', 'Mean_Corpuscular_Volume', 'Meningitis_Contagious', 'Meningoencephalitis_Toxoplasma', 'Migraine', 'Mitral_Valve_Prolapse', 'Multiple_Sclerosis', 'Ophthalmology', 'Peroneal_muscle', 'Posterior_muscles', 'Pulmonary_artery']


In [15]:
#One hot encoding
y=utils.to_categorical(y, num_classes=len(labels))

In [33]:
# tf.convert_to_tensor(y)

**Split the dataset into training and test set

In [16]:
x_tr, x_val, y_tr, y_val = train_test_split(all_wave,y,test_size = 0.2,shuffle=True,random_state=777)

In [17]:
print(x_tr.shape)


(60, 7, 20)


**Normalization of data**

In [18]:
mean = x_tr.mean(axis=0, keepdims=True)
std = x_tr.std(axis=0, keepdims=True)

In [19]:
x_tr = (x_tr - mean) / (std + 1e-8)
x_val = (x_val - mean) / (std + 1e-8)

In [20]:
print(y_tr.shape)

(60, 25)


In [21]:
print(y_val.shape)

(15, 25)


In [22]:
print(x_val.shape)

(15, 7, 20)


**LSTM Model

In [23]:
model1 = Sequential()
model1.add(LSTM(25, input_shape = (x_tr.shape[1:]), activation = "relu"))
model1.add(Dropout(0.5))
model1.add(BatchNormalization())
model1.add(Dense(25, activation = "relu"))
model1.add(Dropout(0.2))
model1.add(Dense(25, activation = "softmax"))

In [24]:
opt1 = keras.optimizers.Adam(learning_rate=0.0001)
model1.compile(loss='categorical_crossentropy',optimizer=opt1,metrics=['accuracy'])

In [26]:
history1=model1.fit(x_tr, y_tr ,epochs=100,batch_size=32, validation_data=(x_val,y_val))

Epoch 1/100
2/2 [==============================] - 0s 28ms/step - loss: 3.0790 - accuracy: 0.0500 - val_loss: 3.2553 - val_accuracy: 0.0667
Epoch 2/100
2/2 [==============================] - 0s 17ms/step - loss: 3.0528 - accuracy: 0.1500 - val_loss: 3.2556 - val_accuracy: 0.0667
Epoch 3/100
2/2 [==============================] - 0s 20ms/step - loss: 2.9591 - accuracy: 0.1500 - val_loss: 3.2547 - val_accuracy: 0.0667
Epoch 4/100
2/2 [==============================] - 0s 17ms/step - loss: 3.0354 - accuracy: 0.0500 - val_loss: 3.2535 - val_accuracy: 0.0667
Epoch 5/100
2/2 [==============================] - 0s 16ms/step - loss: 3.1007 - accuracy: 0.1167 - val_loss: 3.2539 - val_accuracy: 0.0667
Epoch 6/100
2/2 [==============================] - 0s 20ms/step - loss: 3.0652 - accuracy: 0.1000 - val_loss: 3.2527 - val_accuracy: 0.0667
Epoch 7/100
2/2 [==============================] - 0s 20ms/step - loss: 3.1139 - accuracy: 0.1333 - val_loss: 3.2517 - val_accuracy: 0.0667
Epoch 8/100
2/2 [===

**As we can see the accurracy of the data is not increasing due to insufficient data. The basic requirememnt of the neural nets
is they require large amount of data to learn**

In [43]:
#Predictions using LSTM
def predict(audio):
    prob=model1.predict(np.expand_dims(audio, 1))
    index=np.argmax(prob[0])
    print(index)
    return classes[index]

In [44]:
import random
index=random.randint(0,len(x_val)-1)
print(index)
samples=x_val[index]
#print(samples)
print("Audio:",classes[np.argmax(y_val[index])])
#ipd.Audio(samples, rate=8000)

4
Audio: Adenocarcinoma


**Predictions**

In [45]:
print("Text:",predict(samples))

13
Text: Leucoderma


In [43]:
""""
#1D Convolutional Neural Net

K.clear_session()

inputs = Input(shape=(7,20))

#First Conv1D layer
conv = Conv1D(16, 2, padding='same', activation='relu', strides=1)(inputs)
conv = MaxPooling1D(2)(conv)
# conv = Dropout(0.3)(conv)

#Second Conv1D layer
conv = Conv1D(16, 1, padding='same', activation='relu', strides=1)(conv)
conv = GlobalMaxPool1D()(conv)
# conv = Dropout(0.3)(conv)


#Flatten layer
# conv = Flatten()(conv)

#Dense Layer 1
# conv = Dropout(0.5)(conv)
conv = Dense(16, activation='relu')(conv)
conv = Dropout(0.5)(conv)

outputs = Dense(len(labels), activation='softmax')(conv)

model = Model(inputs, outputs)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 7, 20)]           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 7, 16)             656       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 3, 16)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 3, 16)             272       
_________________________________________________________________
global_max_pooling1d (Global (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dropout (Dropout)            (None, 16)               

In [44]:
#opt = keras.optimizers.Adam(learning_rate= 1e-3)
#model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [45]:
#history=model.fit(x_tr, y_tr ,epochs=300,batch_size=8, validation_data=(x_val,y_val))

Epoch 1/300
8/8 [==============================] - 0s 25ms/step - loss: 3.7904 - accuracy: 0.0667 - val_loss: 3.5185 - val_accuracy: 0.0667
Epoch 2/300
8/8 [==============================] - 0s 6ms/step - loss: 3.5327 - accuracy: 0.0167 - val_loss: 3.4258 - val_accuracy: 0.0667
Epoch 3/300
8/8 [==============================] - 0s 6ms/step - loss: 3.5023 - accuracy: 0.0500 - val_loss: 3.3468 - val_accuracy: 0.0667
Epoch 4/300
8/8 [==============================] - 0s 6ms/step - loss: 3.3873 - accuracy: 0.0333 - val_loss: 3.2970 - val_accuracy: 0.0667
Epoch 5/300
8/8 [==============================] - 0s 6ms/step - loss: 3.3205 - accuracy: 0.0833 - val_loss: 3.2616 - val_accuracy: 0.0667
Epoch 6/300
8/8 [==============================] - 0s 6ms/step - loss: 3.2338 - accuracy: 0.0667 - val_loss: 3.2445 - val_accuracy: 0.0667
Epoch 7/300
8/8 [==============================] - 0s 5ms/step - loss: 3.2801 - accuracy: 0.0833 - val_loss: 3.2368 - val_accuracy: 0.0667
Epoch 8/300
8/8 [=========

In [38]:
pip install sounddevice

Note: you may need to restart the kernel to use updated packages.


In [39]:
pip install soundfile

**prompt for users to record voice commands**

In [42]:
import sounddevice as sd
import soundfile as sf

samplerate = 8000  
duration = 3 # seconds
filename = 'migrane.wav'
print("start")
mydata = sd.rec(int(samplerate * duration), samplerate=samplerate,
    channels=1, blocking=True)
print("end")
sd.wait()
sf.write(filename, mydata, samplerate)

start
end
